In [38]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm
from PyRadioLoc.Utils.GeoUtils import GeoUtils
from PyRadioLoc.Pathloss.Models import Cost231HataModel
from PyRadioLoc.Pathloss.Models import OkumuraHataModel
import seaborn as sns

In [24]:
data = pd.read_csv("./databasesTCC/Outdoor_X_train_90%.csv")
#data = pd.read_csv("./databasesTCC/meds_pandasTCC.csv")

In [12]:
LAT_INI  = min(data['lat'])
LAT_INI

-8.05955

In [13]:
LAT_FIN  = max(data['lat'])
LAT_FIN

-8.04642

In [14]:
LONG_INI = min(data['lon'])
LONG_INI

-34.95985

In [15]:
LONG_FIN = max(data['lon'])
LONG_FIN

-34.94511

In [16]:
test = data[data['idx']==6758]
test['lat']

1411   -8.057
Name: lat, dtype: float64

# Gera grids 20x20, 30x30 e 40x40

In [19]:
def generate_grids():
    #LAT_INI  = min(data['lat'])
    #LAT_FIN  = max(data['lat'])
    #LONG_INI = min(data['lon'])
    #LONG_FIN = max(data['lon'])
    
    LAT_INI  = -8.05955
    LAT_FIN  = -8.04642
    LONG_INI = -34.95985
    LONG_FIN = -34.94511
    
    
    delta_x = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_INI, LONG_FIN),
                  GeoUtils.distanceInKm(LAT_FIN, LONG_INI, LAT_FIN, LONG_FIN))
    delta_y = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_FIN, LONG_INI),
                  GeoUtils.distanceInKm(LAT_INI, LONG_FIN, LAT_FIN, LONG_FIN))
    R = [5, 10, 20, 30, 40]               # dimensoes de cada célula m x m

    # Para cada um das dimensoes de R calcula uma matriz de fingerprint e salva e um csv 
    for num in R:    
        n_cells_x = math.ceil((delta_x * 1000) / num)
        n_cells_y = math.ceil((delta_y * 1000) / num)

        inc_lat = (LAT_FIN - LAT_INI) / n_cells_x
        inc_lng = (LONG_FIN - LONG_INI) / n_cells_y

        result = []

        for i in range(n_cells_x):
            for j in range(n_cells_y):
                lat = LAT_INI + i * inc_lat
                lng = LONG_INI + j * inc_lng
                result.append((lat, lng))

        # salva para o CSV
        fingerprint_map = pd.DataFrame(result, columns=['lat', 'lon'])
        fingerprint_map.to_csv("./Grid/grid_{}.csv".format(num), index=False)

        print(n_cells_x, n_cells_y)

In [20]:
generate_grids()

325 291
163 146
82 73
55 49
41 37


In [17]:
Grid = pd.read_csv("./Grid/grid_20.csv")
#sns.jointplot(x='lon', y='lat', data=Grid, height=40)

# Calcular a distância do ponto para as ERBs

In [8]:
def calculate_erb_rssi(fingerprint, erb):
    """
    Calcula todos os valores de rssi de uma erb para todos os pontos de um fingerprint
    """
    #rssi_list = []
    #eirp = erb['Eirp']
    dist_List =  []
    lat_erb = erb['lat']
    lon_erb = erb['lon']
    rssiId = erb['RssiId']

    for n, point in fingerprint.iterrows():
        lat = point['lat']
        lon = point['lon']
        
        distance = GeoUtils.distanceInKm(lat_erb, lon_erb, lat, lon)
        dist_List.append(distance)
    
    return dist_List

def generate_grid(erbs, fingerprint):
    """
    Gera um fingerprint e retorna como dataFrame
    """
    rssi_dict = {}
    for idx, erb in tqdm(erbs.iterrows()):
        rssiId = erb['RssiId']
        rssi_dict[rssiId] = calculate_erb_rssi(fingerprint, erb)
    grid = pd.DataFrame(rssi_dict)
    return grid
    
def fingerprints_to_csv():
    """
    Gera todos os fingerprints e salva em csv
    """
    #dims = [5, 10, 20, 30, 40]
    dims = [5,10]#
    
    erbs = pd.read_csv("./databasesTCC/BtsTCC.csv")
    for dim in tqdm(dims):
        fingerprint = pd.read_csv("./Grid/grid_{}.csv".format(dim))
        grid = generate_grid(erbs, fingerprint)
        grid.insert(0, 'lat', fingerprint['lat'], True)
        grid.insert(1, 'lon', fingerprint['lon'], True)
        grid.rename({"rssi_1_1": "dist_1"}, axis=1, inplace=True)
        grid.rename({"rssi_2_1": "dist_2"}, axis=1, inplace=True)
        grid.rename({"rssi_3_1": "dist_3"}, axis=1, inplace=True)
        grid.to_csv("./CDB/CDB_{}.csv".format(dim), index=False)

In [9]:
# gera os fingerprints que estão em dims
fingerprints_to_csv()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:57, 57.40s/it]
2it [01:54, 57.22s/it]
 50%|█████████████████████████████████████████▌                                         | 1/2 [02:44<02:44, 164.81s/it]
0it [00:00, ?it/s]
1it [00:10, 10.94s/it]
2it [00:21, 10.78s/it]
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [03:16<00:00, 124.99s/it]


# Calcular os delays dos pontos para as ERBs

In [39]:
def PutDelay(df_meds): 

    df_meds["delay_1"] = df_meds["dist_1"].apply(lambda x:int(np.round(x/0.234))) #Dividir por 234 metros
    df_meds["delay_2"] = df_meds["dist_2"].apply(lambda x:int(np.round(x/0.234))) 
    df_meds["delay_3"] = df_meds["dist_3"].apply(lambda x:int(np.round(x/0.234)))
    df_meds["delay_12"] = df_meds["delay_1"] - df_meds["delay_2"]
    df_meds["delay_13"] = df_meds["delay_1"] - df_meds["delay_3"]
    df_meds["delay_23"] = df_meds["delay_2"] - df_meds["delay_3"]
    
    return df_meds

# Calcular os Azimuth das ERBs em relação aos pontos:

In [40]:
def PutAzimuthsFromBTss(df_bts, df_meds):
    
    lat1, lon1 = df_bts['lat'][0], df_bts['lon'][0] # Lat, Lon ERB1
    lat2, lon2 = df_bts['lat'][1], df_bts['lon'][1] # Lat, Lon ERB2
    lat3, lon3 = df_bts['lat'][2], df_bts['lon'][2] # Lat, Lon ERB3
    lats, lons = np.array(df_meds[['lat']]), np.array(df_meds[['lon']])
    
    # Ângulo, Cosseno, Seno e Tagente ERB 1
    df_meds['ang_1'] = GeoUtils.AzimuthAtoB(lat1,lon1,lats,lons)
    df_meds['cos_1'] = np.cos(np.deg2rad(df_meds['ang_1']))
    df_meds['sin_1'] = np.sin(np.deg2rad(df_meds['ang_1']))
    df_meds['tg_1'] =  np.tan(np.deg2rad(df_meds['ang_1']))
    
    # Ângulo, Cosseno, Seno e Tagente ERB 2
    df_meds['ang_2'] = GeoUtils.AzimuthAtoB(lat2,lon2,lats,lons)
    df_meds['cos_2'] = np.cos(np.deg2rad(df_meds['ang_2']))
    df_meds['sin_2'] = np.sin(np.deg2rad(df_meds['ang_2']))
    df_meds['tg_2'] =  np.tan(np.deg2rad(df_meds['ang_2']))
    
    # Ângulo, Cosseno, Seno e Tagente ERB 3
    df_meds['ang_3'] = GeoUtils.AzimuthAtoB(lat3,lon3,lats,lons)
    df_meds['cos_3'] = np.cos(np.deg2rad(df_meds['ang_3']))
    df_meds['sin_3'] = np.sin(np.deg2rad(df_meds['ang_3']))
    df_meds['tg_3'] =  np.tan(np.deg2rad(df_meds['ang_3']))
    
    return df_meds  
    

# Gera Mapa de cobertura (CDB) sem Machine Learning:

In [41]:
df_meds = pd.read_csv("./CDB/CDB_20.csv")
df_bts = pd.read_csv("./databasesTCC/BtsTCC.csv")
df_meds = PutAzimuthsFromBTss(df_bts, df_meds)
df_meds = PutDelay(df_meds)
df_meds.to_csv("./CDB/CDB_20.csv", index=False)

In [67]:
df_meds = pd.read_csv("./CDB/CDB_20_LightGBM_Complete.csv")
df_meds.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lat,lon
0,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959850
1,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959648
2,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959446
3,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959244
4,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,7,2,1,6,5,-8.05955,-34.959042


In [68]:
data = pd.read_csv("./databasesTCC/Outdoor_X_train_90%.csv")
data = data.drop(['idx','dist_1','dist_2','dist_3','ang_1','cos_1','sin_1','tg_1','ang_2','cos_2','sin_2','tg_2','ang_3','cos_3','sin_3','tg_3'],axis = 1)
data_y = pd.read_csv("./databasesTCC/Outdoor_y_train_90%.csv")
data['rssi_1_1'] = data_y['rssi_1_1']
data['rssi_1_2'] = data_y['rssi_1_2']
data['rssi_1_3'] = data_y['rssi_1_3']
data['rssi_2_1'] = data_y['rssi_2_1']
data['rssi_2_2'] = data_y['rssi_2_2']
data['rssi_2_3'] = data_y['rssi_2_3']
data['rssi_3_1'] = data_y['rssi_3_1']
data['rssi_3_2'] = data_y['rssi_3_2']
data['rssi_3_3'] = data_y['rssi_3_3']
data.head()

,lat,lon,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3
0,-8.05858,-34.94940,7,3,4,4,3,-1,-91.801429,-92.775714,-93.930000,-96.246667,-92.3700,-86.497143,-91.915000,-83.340000,-95.8575
1,-8.05213,-34.94860,4,2,5,2,-1,-3,-87.827143,-90.397143,-88.951429,-67.090000,-82.1725,-81.895714,-86.488333,-92.622000,-97.6000
2,-8.05931,-34.94698,8,3,5,5,3,-2,-96.500000,-89.047013,-94.131524,-95.600000,-94.3100,-74.561071,-88.900000,-91.629273,-95.8575
3,-8.04751,-34.95402,2,5,4,-3,-2,1,-76.247143,-88.987143,-77.557500,-86.900000,-84.5200,-91.645000,-80.279444,-102.828000,-97.8275
4,-8.04887,-34.95971,4,7,3,-3,1,4,-75.950000,-88.482857,-74.288571,-86.510000,-91.5200,-93.597143,-84.063333,-104.362000,-81.3025


In [69]:
df_complete = pd.concat([df_meds,data],axis = 0,sort=False)
df_complete = df_complete.sort_values(['lat', 'lon'], ascending=[1,1])
df_complete.to_csv("./CDB/CDB_20_LightGBM_Complete_Med.csv", index=False)
df_complete.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lat,lon
0,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959850
1,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959648
2,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959446
3,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,8,2,0,6,6,-8.05955,-34.959244
4,-88.805625,-92.617442,-88.940553,-88.338837,-90.544829,-89.210798,-87.818557,-91.322734,-93.068943,8,7,2,1,6,5,-8.05955,-34.959042


In [72]:
df_meds_SVR = pd.read_csv("./CDB/CDB_20_SVR_Complete.csv")
df_meds_SVR.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lat,lon
0,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959850
1,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959648
2,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959446
3,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959244
4,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,7,2,1,6,5,-8.05955,-34.959042


In [75]:
df_complete_SVR = pd.concat([df_meds_SVR,data],axis = 0,sort=False)
df_complete_SVR = df_complete_SVR.sort_values(['lat', 'lon'], ascending=[1,1])
df_complete_SVR.to_csv("./CDB/CDB_20_SVR_Complete_Med.csv", index=False)
df_complete_SVR.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lat,lon
0,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959850
1,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959648
2,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959446
3,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,8,2,0,6,6,-8.05955,-34.959244
4,-88.807295,-93.157462,-88.885309,-88.882457,-90.548191,-89.182724,-87.068906,-90.558537,-92.058832,8,7,2,1,6,5,-8.05955,-34.959042


In [42]:
df_data = pd.read_csv("./databasesTCC/meds_pandas.csv")
df_bts = pd.read_csv("./databasesTCC/BtsTCC.csv")
df_data = PutAzimuthsFromBTss(df_bts, df_data)
df_data = PutDelay(df_data)
df_data.to_csv("./databasesTCC/meds_pandasTCCM.csv", index=False)

# Aplica o modelo teorico de Cost231HataModel e gera os fingerprints

In [5]:
FREQ_ERBS = 1800
MODEL = Cost231HataModel(FREQ_ERBS)

In [6]:
def calculate_erb_rssi(fingerprint, erb, model):
    """
    Calcula todos os valores de rssi de uma erb para todos os pontos de um fingerprint
    """
    rssi_list = []
    eirp = erb['Eirp']
    lat_erb = erb['lat']
    lon_erb = erb['lon']
    rssiId = erb['RssiId']

    for n, point in fingerprint.iterrows():
        lat = point['lat']
        lon = point['lon']
        
        distance = GeoUtils.distanceInKm(lat_erb, lon_erb, lat, lon)
        pathloss = model.pathloss(distance)
        rssi = eirp - pathloss
        rssi_list.append(rssi)
    
    return rssi_list

def generate_grid(erbs, fingerprint):
    """
    Gera um fingerprint e retorna como dataFrame
    """
    rssi_dict = {}
    for idx, erb in tqdm(erbs.iterrows()):
        rssiId = erb['RssiId']
        rssi_dict[rssiId] = calculate_erb_rssi(fingerprint, erb, MODEL)
    grid = pd.DataFrame(rssi_dict)
    return grid
    
def fingerprints_to_csv():
    """
    Gera todos os fingerprints e salva em csv
    """
    dims = [5, 10, 20, 30, 40]
    erbs = pd.read_csv("./databases/Bts.csv")
    for dim in tqdm(dims):
        fingerprint = pd.read_csv("./Grid/grid_{}.csv".format(dim))
        grid = generate_grid(erbs, fingerprint)
        grid.insert(0, 'lat', fingerprint['lat'], True)
        grid.insert(1, 'lon', fingerprint['lon'], True)
        grid.to_csv("./Fingerprint/fingerprint_{}.csv".format(dim), index=False)

In [7]:
# gera os fingerprints que estão em dims
fingerprints_to_csv()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:48, 48.72s/it]
2it [01:37, 48.71s/it]
3it [02:26, 48.95s/it]
4it [03:10, 47.25s/it]
5it [03:57, 47.17s/it]
6it [04:43, 47.05s/it]
7it [05:28, 46.25s/it]
8it [06:12, 45.49s/it]
 20%|████████████████▌                                                                  | 1/5 [07:00<28:01, 420.40s/it]
0it [00:00, ?it/s]
1it [00:11, 11.79s/it]
2it [00:23, 11.78s/it]
3it [00:34, 11.64s/it]
4it [00:46, 11.51s/it]
5it [00:57, 11.47s/it]
6it [01:08, 11.41s/it]
7it [01:19, 11.31s/it]
8it [01:30, 11.23s/it]
 40%|█████████████████████████████████▏                                                 | 2/5 [08:43<16:15, 325.16s/it]
0it [00:00, ?it/s]
1it [00:02,  2.99s/it]
2it [00:05,  2.99s/it]
3it [00:08,  2.98s/it]
4it [00:11,  2.95s/it]
5it [00:14,  2.92s/it]
6it [00:17,  2.89s/it]
7it [00:20,  2.86s/it]
8it [00:22,  2.79s/it]
 60%|██████████████████████████